# Анализ базы данных сервиса чтения книг по подписке

***Цель исследования*** Моя компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Исследование таблиц

In [13]:
import pandas as pd
import sqlalchemy as sa


In [14]:
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})

In [15]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    """Открываем соединение, получаем данные из sql, закрываем соединение"""
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

Настройка соединения к базе данных и функция для запросов данных

In [8]:
query = '''SELECT * FROM books LIMIT 5'''
print('Таблица books:')
display(get_sql_data(query))
query = '''SELECT COUNT(*) FROM books '''
print('Количество записей в таблице:\n',get_sql_data(query))

Таблица books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество записей в таблице:
    count
0   1000


In [9]:
query = '''SELECT * FROM authors LIMIT 5'''
print('Таблица authors:')
display(get_sql_data(query))
query = '''SELECT COUNT(*) FROM authors '''
print('Количество записей в таблице:\n',get_sql_data(query))

Таблица authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Количество записей в таблице:
    count
0    636


In [10]:
query = '''SELECT * FROM ratings LIMIT 5'''
print('Таблица ratings:')
display(get_sql_data(query))
query = '''SELECT COUNT(*) FROM ratings '''
print('Количество записей в таблице:\n',get_sql_data(query))

Таблица ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество записей в таблице:
    count
0   6456


In [11]:
query = '''SELECT * FROM reviews LIMIT 5'''
print('Таблица reviews:')
display(get_sql_data(query))
query = '''SELECT COUNT(*) FROM reviews '''
print('Количество записей в таблице:\n',get_sql_data(query))

Таблица reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество записей в таблице:
    count
0   2793


In [12]:
query = '''SELECT * FROM publishers LIMIT 5'''
print('Таблица publishers:')
display(get_sql_data(query))
query = '''SELECT COUNT(*) FROM publishers '''
print('Количество записей в таблице:\n',get_sql_data(query))

Таблица publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Количество записей в таблице:
    count
0    340


### Подсчет , сколько книг вышло после 1 января 2000 года

In [34]:
query = '''SELECT COUNT(book_id) 
FROM books
WHERE  CAST(publication_date as date) > '2000-01-01'  '''
display(get_sql_data(query))

,count
0,819


819 книг вышло после 1 января 2000 года

### Для каждой книги подсчет количества обзоров и средней оценки

In [19]:
query = '''SELECT bo.title, COUNT(DISTINCT re.review_id),AVG(ra.rating)  
FROM books as bo LEFT JOIN reviews as re ON bo.book_id = re.book_id LEFT JOIN ratings as ra ON bo.book_id = ra.book_id
GROUP BY bo.book_id
ORDER BY COUNT(DISTINCT re.review_id) DESC
LIMIT 5 '''
display(get_sql_data(query))

,title,count,avg
0,Twilight (Twilight #1),7,3.662500
1,Outlander (Outlander #1),6,4.125000
2,Eat Pray Love,6,3.395833
3,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
4,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634


Подсчитано количество обзоров и средняя оценка для каждой книги

### Определение издательства, которое выпустило наибольшее число книг толще 50 страниц — так исключаются из анализа брошюры

In [20]:
query = '''WITH a AS
(SELECT pu.publisher , COUNT(bo.book_id) OVER (PARTITION BY pu.publisher_id) AS count_books  
FROM books as bo LEFT JOIN publishers as pu ON bo.publisher_id = pu.publisher_id 
WHERE bo.num_pages > 50
ORDER BY count_books DESC 
LIMIT 1)
SELECT publisher 
FROM a'''
display(get_sql_data(query))

,publisher
0,Penguin Books


Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц

### Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками

In [23]:
query = '''
WITH a1 AS
(SELECT bo.book_id ,bo.author_id as au_id, AVG(rating) AS avg_book
FROM books as bo LEFT JOIN ratings as ra ON bo.book_id = ra.book_id 
GROUP BY bo.book_id
HAVING COUNT(rating)>49),
a2 AS (SELECT au_id, AVG(avg_book) OVER (PARTITION BY au_id) AS avg__ra_books 
FROM a1
ORDER BY avg__ra_books DESC
LIMIT 1)
SELECT author
FROM a2 INNER JOIN authors  ON a2.au_id = authors.author_id
'''
display(get_sql_data(query))

,author
0,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré  - это авторы книг о Гарри Поттере получилис самую высокую оценку книг

### Подсчет среднего количество обзоров от пользователей, которые поставили больше 48 оценок

In [24]:
query = '''
WITH a1 AS
(SELECT COUNT(review_id) AS count_reviews
FROM reviews
WHERE username IN (SELECT username  FROM ratings GROUP BY username HAVING COUNT(rating)>48)
GROUP BY username)
SELECT AVG(count_reviews)
FROM a1

 '''
display(get_sql_data(query))

,avg
0,24.0


24 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок